# Bitcoin Transactions from Google API

INFO: Data stops at 2018-09-10.

In [ ]:
import os

import pandas as pd

import matplotlib
import matplotlib.pyplot
import matplotlib.patheffects

import style
style.use_dark_theme()

from google.cloud import bigquery

In [ ]:
# Replace this with your Google API credential
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.expanduser('~/Downloads/bitcoin-analytics-296223-24d70d86e816.json')

In [ ]:
# filename = os.path.expanduser('~/Downloads/btc-trns.pkl')
filename = 'blob/btc-trns.pkl'
if os.path.exists(filename):
    transactions = pd.read_pickle(filename)
else:
    client = bigquery.Client()

    # Query by Allen Day, GooglCloud Developer Advocate (https://medium.com/@allenday)
    query = """
    #standardSQL
    SELECT
      o.day,
      COUNT(DISTINCT(o.output_key)) AS recipients
    FROM (
      SELECT
        TIMESTAMP_MILLIS((timestamp - MOD(timestamp, 86400000))) AS day,
        output.output_pubkey_base58 AS output_key
      FROM
        `bigquery-public-data.bitcoin_blockchain.transactions`,
        UNNEST(outputs) AS output ) AS o
    GROUP BY
      day
    ORDER BY
      day
    """

    query_job = client.query(query)

    iterator = query_job.result(timeout=30)
    rows = list(iterator)

    # Transform the rows into a nice pandas dataframe
    transactions = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

    print('Saving a copy ...')
    transactions.to_pickle(filename)

In [ ]:
transactions.head(10)

In [ ]:
x = matplotlib.dates.date2num(transactions.iloc[:, 0])
y = transactions.iloc[:, 1]

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(9, 4.5), dpi=160)
ax = matplotlib.pyplot.axes([0.06, 0.08, 0.92, 0.82])
ax.tick_params(axis='y')
ax.plot(x, y)

ax.grid()

years = matplotlib.dates.YearLocator()   # every year
months = matplotlib.dates.MonthLocator()  # every month
years_fmt = matplotlib.dates.DateFormatter('%Y')

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

ax.set_xlabel('Date')
ax.set_ylabel('Transactions')
title_text = ax.set_title('BTC Transactions', fontweight='bold', fontsize=16)
title_text.set_path_effects([
    matplotlib.patheffects.Stroke(linewidth=1, foreground=(0, 0, 0, 0.7)),
    matplotlib.patheffects.Normal()
])

In [ ]:
transactions.iloc[:, 0]